# Análisis de Agrupación para Datos de Samsung

Este notebook implementa un algoritmo de agrupación (clustering) para analizar patrones en el conjunto de datos de Samsung. Se incluye una justificación del algoritmo, descripción del diseño, evaluación del modelo, así como visualizaciones personalizadas e interpretación de resultados.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
from mpl_toolkits.mplot3d import Axes3D
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('dark_background')
sns.set_palette('magma')
plt.rcParams['figure.figsize'] = (14, 9)
plt.rcParams['font.size'] = 12

## 1. Carga y Exploración de Datos

Primero, cargamos el conjunto de datos de Samsung y exploramos su estructura.

In [ ]:
# Cargar el dataset
df = pd.read_csv('samsung.csv')

# Mostrar las primeras filas
print('Primeras 5 filas del dataset:')
display(df.head())

# Información del dataset
print('
Información del dataset:')
df.info()

# Estadísticas descriptivas
print('
Estadísticas descriptivas:')
display(df.describe())

# Verificar valores nulos
print('
Valores nulos por columna:')
print(df.isnull().sum())

# Eliminar filas con valores nulos si existen
df_clean = df.dropna()
print(f'
Filas originales: {df.shape[0]}, Filas después de eliminar nulos: {df_clean.shape[0]}')

## 2. Análisis Exploratorio de Datos (EDA)

Realizamos un análisis exploratorio para entender mejor las distribuciones y relaciones entre las variables.

In [ ]:
# Seleccionar solo columnas numéricas para el análisis
numeric_df = df_clean.select_dtypes(include=[np.number])

# Matriz de correlación
plt.figure(figsize=(16, 14))
correlation_matrix = numeric_df.corr()

# Crear una máscara para el triángulo superior
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Generar un mapa de calor con la máscara
sns.heatmap(correlation_matrix, mask=mask, annot=False, cmap='magma', linewidths=0.5, vmin=-1, vmax=1)
plt.title('Matriz de Correlación (Triángulo Inferior)', fontsize=18)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Distribución de variables
# Seleccionar un subconjunto de columnas si hay demasiadas
if numeric_df.shape[1] > 10:
    # Seleccionar las 10 columnas con mayor varianza
    variances = numeric_df.var().sort_values(ascending=False)
    top_columns = variances.index[:10].tolist()
    plot_df = numeric_df[top_columns]
else:
    plot_df = numeric_df

# Crear histogramas para cada variable
fig, axes = plt.subplots(nrows=int(np.ceil(plot_df.shape[1]/3)), ncols=3, figsize=(18, 15))
axes = axes.flatten()

for i, column in enumerate(plot_df.columns):
    if i < len(axes):
        sns.histplot(plot_df[column], kde=True, ax=axes[i], color='magenta')
        axes[i].set_title(f'Distribución de {column}', fontsize=14)
        axes[i].set_xlabel(column, fontsize=12)
        axes[i].set_ylabel('Frecuencia', fontsize=12)

# Ocultar ejes vacíos
for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

# Gráfico de dispersión de pares para las variables con mayor varianza
if numeric_df.shape[1] > 5:
    # Seleccionar las 5 columnas con mayor varianza
    top5_columns = variances.index[:5].tolist()
    pair_plot_df = numeric_df[top5_columns]
else:
    pair_plot_df = numeric_df

plt.figure(figsize=(16, 12))
sns.pairplot(pair_plot_df, diag_kind='kde', plot_kws={'alpha': 0.6, 's': 80, 'edgecolor': 'k'})
plt.suptitle('Gráficos de Dispersión entre Pares de Variables', y=1.02, fontsize=16)
plt.tight_layout()
plt.show()

# Análisis de outliers con boxplots
plt.figure(figsize=(18, 10))

# Crear un boxplot para cada variable
ax = sns.boxplot(data=plot_df, palette='magma')
plt.title('Detección de Outliers por Variable', fontsize=16)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## 3. Justificación del Algoritmo

Para este problema de agrupación no supervisada, he seleccionado el algoritmo K-means como método principal por las siguientes razones:

1. **Eficiencia computacional**: K-means es un algoritmo eficiente que escala bien con conjuntos de datos grandes, lo que es importante para analizar datos de dispositivos Samsung que pueden ser voluminosos.

2. **Interpretabilidad**: Los centroides de K-means proporcionan una representación clara y comprensible de cada grupo, facilitando la interpretación de los patrones identificados.

3. **Flexibilidad en la forma de los clusters**: Aunque K-means asume clusters esféricos, esta limitación puede mitigarse mediante la transformación adecuada de los datos o el uso de técnicas de reducción de dimensionalidad como PCA antes de aplicar el algoritmo.

4. **Facilidad para determinar el número óptimo de clusters**: Existen métodos bien establecidos como el método del codo, la puntuación de silueta y el índice Davies-Bouldin para determinar el número óptimo de clusters.

5. **Aplicabilidad a datos de comportamiento de usuario**: K-means es particularmente útil para segmentar usuarios basándose en patrones de comportamiento, lo que es relevante para datos de dispositivos Samsung.

Además, compararemos K-means con otros algoritmos de clustering (DBSCAN y Agglomerative Clustering) para validar nuestra elección y explorar diferentes perspectivas de agrupación.

## 4. Preparación de Datos para Clustering

In [ ]:
# Seleccionar las variables para el clustering
# Usar todas las variables numéricas o un subconjunto según el análisis exploratorio
X = numeric_df.copy()

# Escalar los datos - Probar diferentes escaladores
scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler()
}

scaled_data = {}
for name, scaler in scalers.items():
    scaled_data[name] = scaler.fit_transform(X)
    
# Por defecto, usaremos StandardScaler para el análisis principal
X_scaled = scaled_data['StandardScaler']

print(f'Dimensiones de los datos escalados: {X_scaled.shape}')

# Reducción de dimensionalidad con PCA para visualización
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Varianza explicada
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print('
Varianza explicada por componente:')
for i, var in enumerate(explained_variance):
    print(f'Componente {i+1}: {var:.4f} ({cumulative_variance[i]:.4f} acumulado)')

# Visualizar la varianza explicada
plt.figure(figsize=(12, 6))
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.8, color='magenta', label='Varianza individual')
plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='Varianza acumulada', color='cyan')
plt.axhline(y=0.8, color='r', linestyle='--', label='Umbral 80%')
plt.xlabel('Número de Componentes', fontsize=14)
plt.ylabel('Ratio de Varianza Explicada', fontsize=14)
plt.title('Varianza Explicada por Componentes Principales', fontsize=16)
plt.legend(loc='best', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Visualización de datos en espacio PCA 3D
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c='magenta', s=50, alpha=0.6, edgecolors='w')

ax.set_title('Visualización 3D de Datos con PCA', fontsize=16)
ax.set_xlabel(f'PC1 ({explained_variance[0]:.2%})', fontsize=14)
ax.set_ylabel(f'PC2 ({explained_variance[1]:.2%})', fontsize=14)
ax.set_zlabel(f'PC3 ({explained_variance[2]:.2%})', fontsize=14)

# Añadir una cuadrícula
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Visualización interactiva con Plotly (opcional)
try:
    fig = px.scatter_3d(
        x=X_pca[:, 0], y=X_pca[:, 1], z=X_pca[:, 2],
        opacity=0.7,
        title='Visualización 3D Interactiva de Datos con PCA',
        labels={'x': f'PC1 ({explained_variance[0]:.2%})',
                'y': f'PC2 ({explained_variance[1]:.2%})',
                'z': f'PC3 ({explained_variance[2]:.2%})'}
    )
    
    fig.update_traces(marker=dict(size=5, line=dict(width=1, color='white')))
    fig.update_layout(
        template='plotly_dark',
        scene=dict(
            xaxis=dict(showbackground=False),
            yaxis=dict(showbackground=False),
            zaxis=dict(showbackground=False)
        )
    )
    
    fig.show()
except:
    print('Plotly no está disponible o no se pudo generar la visualización interactiva.')

## 5. Descripción del Diseño del Modelo

Implementaremos un enfoque sistemático para el clustering:

1. **Determinación del número óptimo de clusters**: Utilizaremos múltiples métodos (método del codo, puntuación de silueta, índice Davies-Bouldin) para determinar el número óptimo de clusters.

2. **Aplicación de K-means**: Implementaremos K-means con el número óptimo de clusters determinado.

3. **Evaluación del modelo**: Evaluaremos la calidad del clustering utilizando métricas internas como la puntuación de silueta y el índice Davies-Bouldin.

4. **Comparación con otros algoritmos**: Compararemos K-means con DBSCAN y Agglomerative Clustering para validar nuestra elección.

5. **Interpretación de clusters**: Analizaremos las características de cada cluster para entender los patrones identificados.

In [ ]:
# 1. Determinación del número óptimo de clusters

# Método del codo
inertia_values = []
silhouette_scores = []
davies_bouldin_scores = []
calinski_harabasz_scores = []

k_range = range(2, 15)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia_values.append(kmeans.inertia_)
    
    # Calcular métricas de evaluación
    labels = kmeans.labels_
    silhouette_scores.append(silhouette_score(X_scaled, labels))
    davies_bouldin_scores.append(davies_bouldin_score(X_scaled, labels))
    calinski_harabasz_scores.append(calinski_harabasz_score(X_scaled, labels))

# Visualizar resultados
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Método del codo (Inertia)
axes[0, 0].plot(k_range, inertia_values, marker='o', linestyle='-', linewidth=2, markersize=8, color='magenta')
axes[0, 0].set_title('Método del Codo (Inertia)', fontsize=16)
axes[0, 0].set_xlabel('Número de Clusters (k)', fontsize=14)
axes[0, 0].set_ylabel('Inertia', fontsize=14)
axes[0, 0].grid(True, alpha=0.3)

# Puntuación de Silueta
axes[0, 1].plot(k_range, silhouette_scores, marker='o', linestyle='-', linewidth=2, markersize=8, color='cyan')
axes[0, 1].set_title('Puntuación de Silueta', fontsize=16)
axes[0, 1].set_xlabel('Número de Clusters (k)', fontsize=14)
axes[0, 1].set_ylabel('Puntuación de Silueta', fontsize=14)
axes[0, 1].grid(True, alpha=0.3)

# Índice Davies-Bouldin
axes[1, 0].plot(k_range, davies_bouldin_scores, marker='o', linestyle='-', linewidth=2, markersize=8, color='yellow')
axes[1, 0].set_title('Índice Davies-Bouldin', fontsize=16)
axes[1, 0].set_xlabel('Número de Clusters (k)', fontsize=14)
axes[1, 0].set_ylabel('Índice Davies-Bouldin', fontsize=14)
axes[1, 0].grid(True, alpha=0.3)

# Índice Calinski-Harabasz
axes[1, 1].plot(k_range, calinski_harabasz_scores, marker='o', linestyle='-', linewidth=2, markersize=8, color='lime')
axes[1, 1].set_title('Índice Calinski-Harabasz', fontsize=16)
axes[1, 1].set_xlabel('Número de Clusters (k)', fontsize=14)
axes[1, 1].set_ylabel('Índice Calinski-Harabasz', fontsize=14)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Determinar el número óptimo de clusters
# Para el método del codo, buscamos el punto de inflexión
# Para la puntuación de silueta, buscamos el máximo
# Para el índice Davies-Bouldin, buscamos el mínimo
# Para el índice Calinski-Harabasz, buscamos el máximo

optimal_k_silhouette = k_range[np.argmax(silhouette_scores)]
optimal_k_davies = k_range[np.argmin(davies_bouldin_scores)]
optimal_k_calinski = k_range[np.argmax(calinski_harabasz_scores)]

print(f'Número óptimo de clusters según la puntuación de silueta: {optimal_k_silhouette}')
print(f'Número óptimo de clusters según el índice Davies-Bouldin: {optimal_k_davies}')
print(f'Número óptimo de clusters según el índice Calinski-Harabasz: {optimal_k_calinski}')

# Visualización de silueta para el número óptimo de clusters
optimal_k = optimal_k_silhouette  # Podemos elegir cualquiera de los tres o hacer una votación

# Crear un modelo K-means con el número óptimo de clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

# Visualizar la silueta
plt.figure(figsize=(12, 8))
visualizer = SilhouetteVisualizer(kmeans, colors='magma', is_fitted=True)
visualizer.fit(X_scaled)
visualizer.show()

# Añadir los clusters al DataFrame original
df_clean['Cluster'] = cluster_labels

# Mostrar el tamaño de cada cluster
cluster_sizes = df_clean['Cluster'].value_counts().sort_index()
print('
Tamaño de cada cluster:')
for cluster, size in cluster_sizes.items():
    print(f'Cluster {cluster}: {size} muestras ({size/len(df_clean):.2%})')

## 6. Visualización de Clusters e Interpretación

In [ ]:
# Visualización de clusters en espacio PCA
# Aplicar PCA a los datos escalados
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Crear un DataFrame con los componentes principales y los clusters
pca_df = pd.DataFrame(
    data=X_pca,
    columns=['PC1', 'PC2', 'PC3']
)
pca_df['Cluster'] = cluster_labels

# Visualización 3D de clusters
fig = plt.figure(figsize=(14, 12))
ax = fig.add_subplot(111, projection='3d')

# Crear un mapa de colores personalizado
colors = plt.cm.magma(np.linspace(0, 1, optimal_k))

for cluster in range(optimal_k):
    # Filtrar puntos del cluster actual
    cluster_points = pca_df[pca_df['Cluster'] == cluster]
    
    # Graficar puntos
    ax.scatter(
        cluster_points['PC1'],
        cluster_points['PC2'],
        cluster_points['PC3'],
        s=50,
        color=colors[cluster],
        label=f'Cluster {cluster}',
        alpha=0.7,
        edgecolors='w'
    )
    
    # Graficar centroides (transformados al espacio PCA)
    centroid_scaled = kmeans.cluster_centers_[cluster].reshape(1, -1)
    centroid_pca = pca.transform(centroid_scaled)
    
    ax.scatter(
        centroid_pca[:, 0],
        centroid_pca[:, 1],
        centroid_pca[:, 2],
        s=200,
        color=colors[cluster],
        marker='*',
        edgecolors='k',
        linewidth=2,
        alpha=1.0
    )

ax.set_title(f'Visualización 3D de Clusters (K={optimal_k})', fontsize=16)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})', fontsize=14)
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})', fontsize=14)
ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2]:.2%})', fontsize=14)

# Añadir leyenda
ax.legend(title='Clusters', title_fontsize=12, fontsize=10, loc='best')

# Añadir una cuadrícula
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Visualización interactiva con Plotly (opcional)
try:
    # Crear un DataFrame para Plotly
    plotly_df = pca_df.copy()
    plotly_df['Cluster'] = plotly_df['Cluster'].astype(str)
    
    # Crear la figura 3D
    fig = px.scatter_3d(
        plotly_df,
        x='PC1',
        y='PC2',
        z='PC3',
        color='Cluster',
        opacity=0.7,
        title=f'Visualización 3D Interactiva de Clusters (K={optimal_k})',
        labels={'PC1': f'PC1 ({pca.explained_variance_ratio_[0]:.2%})',
                'PC2': f'PC2 ({pca.explained_variance_ratio_[1]:.2%})',
                'PC3': f'PC3 ({pca.explained_variance_ratio_[2]:.2%})'}
    )
    
    # Añadir centroides
    for cluster in range(optimal_k):
        centroid_scaled = kmeans.cluster_centers_[cluster].reshape(1, -1)
        centroid_pca = pca.transform(centroid_scaled)
        
        fig.add_trace(
            go.Scatter3d(
                x=centroid_pca[:, 0],
                y=centroid_pca[:, 1],
                z=centroid_pca[:, 2],
                mode='markers',
                marker=dict(
                    size=15,
                    symbol='star',
                    color=cluster,
                    line=dict(width=2, color='black')
                ),
                name=f'Centroide {cluster}'
            )
        )
    
    fig.update_layout(
        template='plotly_dark',
        scene=dict(
            xaxis=dict(showbackground=False),
            yaxis=dict(showbackground=False),
            zaxis=dict(showbackground=False)
        )
    )
    
    fig.show()
except:
    print('Plotly no está disponible o no se pudo generar la visualización interactiva.')

# Visualización 2D de clusters (PC1 vs PC2)
plt.figure(figsize=(14, 10))

for cluster in range(optimal_k):
    # Filtrar puntos del cluster actual
    cluster_points = pca_df[pca_df['Cluster'] == cluster]
    
    # Graficar puntos
    plt.scatter(
        cluster_points['PC1'],
        cluster_points['PC2'],
        s=80,
        color=colors[cluster],
        label=f'Cluster {cluster}',
        alpha=0.7,
        edgecolors='w'
    )
    
    # Graficar centroides (transformados al espacio PCA)
    centroid_scaled = kmeans.cluster_centers_[cluster].reshape(1, -1)
    centroid_pca = pca.transform(centroid_scaled)
    
    plt.scatter(
        centroid_pca[:, 0],
        centroid_pca[:, 1],
        s=200,
        color=colors[cluster],
        marker='*',
        edgecolors='k',
        linewidth=2,
        alpha=1.0
    )

plt.title(f'Visualización 2D de Clusters (K={optimal_k})', fontsize=16)
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})', fontsize=14)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(title='Clusters', title_fontsize=12, fontsize=10, loc='best')
plt.tight_layout()
plt.show()